In [ ]:
%pip install dremio-simple-query

In [2]:
from dremio_simple_query.connect import get_token

## URL to Login Endpoint
login_endpoint = "http://localhost:9047/apiv2/login"

## Payload for Login
## Update with your Dremio credentials
payload = {
    "userName": "username",
    "password": "password"
}

## Get token from API
token = get_token(uri = login_endpoint, payload=payload)

In [ ]:
from pyarrow import flight

# Connect to Dremio using Flight
client = flight.FlightClient(f"grpc+tcp://localhost:32010")

# Authenticate
headers = [
    (b'authorization', f"bearer {token}".encode('utf-8'))
]

# Query to get all product reviews
sql = "SELECT * FROM catalog.silver.vehicle_health_logs"

# Create a Flight descriptor
descriptor = flight.FlightDescriptor.for_command(sql)

# Perform the query using call_options to include headers
options = flight.FlightCallOptions(headers=headers)
flight_info = client.get_flight_info(descriptor, options=options)
reader = client.do_get(flight_info.endpoints[0].ticket, options=options)

# Convert to Pandas DataFrame
vehicle_health_logs = reader.read_pandas()

# Display the DataFrame
vehicle_health_logs.head()

In [ ]:
import pandas as pd
from pandas import json_normalize

# Flatten MaintenanceHistory
maintenance_records = json_normalize(vehicle_health_logs.explode('MaintenanceHistory')['MaintenanceHistory'])

# Analysis: Maintenance Frequency
maintenance_frequency = maintenance_records['Type'].value_counts()
print("Maintenance Frequency:")
print(maintenance_frequency)


In [ ]:
# Analysis: Outcome Analysis
outcome_analysis = maintenance_records['Outcome'].value_counts()
print("\nOutcome Analysis:")
print(outcome_analysis)

In [ ]:
# Flatten Alerts
alerts_records = json_normalize(vehicle_health_logs.explode('Alerts')['Alerts'])

# Analysis: Alerts Analysis
alerts_analysis = alerts_records['Type'].value_counts()
print("\nAlerts Analysis:")
print(alerts_analysis)

In [ ]:
import matplotlib.pyplot as plt

# Flatten Alerts
alerts_records = json_normalize(vehicle_health_logs.explode('Alerts')['Alerts'])

# Plot alert types
alert_types = alerts_records['Type'].value_counts()
alert_types.plot(kind='bar', figsize=(8, 6))
plt.title('Alert Types Distribution')
plt.xlabel('Alert Type')
plt.ylabel('Frequency')
plt.show()

# Plot alert resolutions
alert_resolutions = alerts_records['Resolved'].value_counts()
alert_resolutions.plot(kind='bar', figsize=(8, 6))
plt.title('Alert Resolutions')
plt.xlabel('Resolved')
plt.ylabel('Frequency')
plt.xticks([0, 1], ['False', 'True'])
plt.show()